In [54]:
import matplotlib.pyplot as plt
import pandas as pd
import os 

In [51]:
rawpath = './Raw Data'

filenames = os.listdir(rawpath)
filenames.sort()

stationA = pd.DataFrame(columns=['time','batt_num'])
stationB = pd.DataFrame(columns=['time','batt_num'])

for f in filenames:
    if 'B' in f:
        raw = pd.read_csv(rawpath+'/'+f)
        stationB = pd.concat([stationB,raw])
    elif 'A' in f:
        raw = pd.read_csv(rawpath+'/'+f)
        stationA = pd.concat([stationA,raw])
    else:
        continue
    

In [ ]:
plt.figure()
plt.plot(stationA['time'],stationA['batt_num'])